***Задание №1***

Спроектировать объектную модель сервиса:
* Выделить базовые сущности с которыми будет происходить работа. Пользователь, ML модель, история транзакций/предсказаний, задача для ML модели и т.д.
* При проектирование написать названия классов, названия полей, методов, принимаемых типов данных, модификаторов доступа.
* При проектирование использовать все базовые принципы ООП, а именно инкапсуляцию, наследование для расширения классов и полиморфизм.

In [275]:
import numpy as np
import pandas as pd
import datetime
import random

## ML-модель

In [584]:
# Простой класс для хранения модели и ее обще описательных характеристик

class MLmodel:
    def __init__(self, model, name, version, description):
        self.model = model # сама модель
        self.name = name # Название модели
        self.version = version # Версия модели
        self.description = description # Описательная характеристика модели

    def __str__(self):
        print(f"{self.model} \n" \
               f"Название: {self.name} \n"  \
               f"Версия: {self.version} \n" \
               f"Описание: {self.description}")

In [590]:
# Наследуемый класс модели для учета ее параметров использования

class UsableModel(MLmodel):
    def __init__(self, model, name, version, description, price):
        super().__init__(model, name, version, description)
        self.price = price # Стоимость использования модели

        self.StatusOperation = None # Статус последнего запуска
        self.Result = None # Результаты работы модели

    def run(self):
        # Запускаем модель и сообщаем о статусе
        self.StatusOperation = bool(random.randint(0, 1)) # Успешность модели пока рандомна - может работать, а может нет
        print('| Модель сработала: ', self.StatusOperation, '|\n' )
        self.Result = 'Результаты работы модели'    

    def __str__(self):
        super().__str__()
        print(f"Стоимость: {self.price} \n"  \
              f"Статус последнего запуска: {self.StatusOperation}") 

In [592]:
CurrModel_1 = UsableModel('', 'Убиватор-3000', '1.01', 'Офигенская модель, прям всем советую только ее',
                          price = 1)

CurrModel_2 = UsableModel('', 'УльтраУбиватор-9000', '3.33', 'Еще более крутая и дорогая модель.',
                          price = 2)

## Пользователь

In [712]:
class User:

    def __init__(self, name, email, startBallance, FreeLimitPerDay):
        self.name = name # имя пользователя
        self.email = email # электронный адрес для взаимодействия
        self.RegDate = datetime.datetime.now() # Дата-время регистрации
        self.role = 0 # Роль, по умолчанию "пользователь"
        
        self.rang = 0 # Ранг пользоватля (может влиять на количество бесплатных операций)
        self.FreeLimitPerDay = FreeLimitPerDay # Количество бесплатных операций в сутки

        self.ballance = startBallance # Стартовый балланс
        self.LastOperationDate = np.NaN # Дата последней удачной операции
        self.HistoryOperation = pd.DataFrame(columns=['Date', 'Operation', 'Success']) # Список всех операций и дата свершения
        
        self.OperationAll = 0 # Сколько операций сделано за все время
        self.OperationToday = 0 # сколько операций сделано сегодня
        self.FreeLimitToday = FreeLimitPerDay # сколько обесплатных операций осталось на сегодня

        self.GetFirstOperation = False # Прошел ли пользователь инициацию первым успешным использованием сервисв
        

    # Операция пополнения балланса    
    def refund(self, payment):
        self.ballance += payment
        print(f"Пополнение баланса на {payment} ед. прошлошло успешно. Текущий балланс — {self.ballance} ед.")


    # Операция возобновления суточных лимитов
    def refillLimits(self, Num = None):
        # если Num == 0, значит возобновим лимит полностью
        # во всех остальных случаях делаем +Num
        if Num == None:
            self.FreeLimitToday = self.FreeLimitPerDay
        else:
            self.FreeLimitToday += Num
        
        print(f"Лимиты бесплатных вызовов в день обновлены. Сейчас доступно — {self.FreeLimitToday}")


    # Операция получения балланса пользователя
    def get_ballance(self):
        return self.ballance 


    # Оплата операции
    def pay(self, payment):
        ResultOperation = False # все плохо, если ничто это го не изменит

        # Если пользователь не исчерпал бесплатные операции на день - он ничего не платит
        # Если исчерпал - платит
        if self.FreeLimitToday > 0:
            self.FreeLimitToday -= 1
            ResultOperation = True
        else:    
            if self.ballance - payment >= 0:
                self.ballance -= payment
                ResultOperation = True
            else:
                print(f"Пополните баланс или дождитесь возобновления суточного лимита обращений. \nДля выполнения операции не хватает {abs(self.ballance - payment)} ед.")
            
        return ResultOperation

    
    # Операция обновления истории действий пользователя
    def updateHistory(self, Date, Operation, Success):
        LastPos = len(self.HistoryOperation)
        self.HistoryOperation.loc[LastPos, 'Date'] = Date
        self.HistoryOperation.loc[LastPos, 'Operation'] = Operation
        self.HistoryOperation.loc[LastPos, 'Success'] = Success



    def useModel(self, payment, model: MLmodel):
        #Процедура запуска модели, с проверкой возможности запуска, сохранением параметров и списания баланса
        ResultOperation = False # все плохо, если ничто это го не изменит

        if self.pay(payment):
            # Если платеж прошел или мы можем провести операцию бесплатно в рамках лимита
            model.run()
            self.updateHistory(Date = datetime.datetime.now(),
                    Operation = 'Запуск модели',
                    Success = model.StatusOperation)



            if not model.StatusOperation:
                # Если запуск модели был неудачный, нужно вернуть оплату 
                # Если операция шла за счет бесплатных, то вернем клиенту это деньгами (надеюсь, нас не обонкротят :О))
                self.refund(payment)
            else:
                ResultOperation = True
                
                self.LastOperationDate = datetime.datetime.now()
                self.OperationToday += 1
                self.OperationAll += 1

                if not self.GetFirstOperation:
                    # Если пользователь совершил свою самею первую операцию - отметим этом
                    self.GetFirstOperation = True

                    # И вообще, можем его как то поздравить!

        return ResultOperation

In [511]:
# Функция вывода профиля пользователя
def showUserDetails(CurrentUser):
    print('Имя пользователя = ', CurrentUser.name)
    print('Эл. почта = ', CurrentUser.email)
    print('Дата регистрации = ', CurrentUser.RegDate)
    print('Роль = ', CurrentUser.role)
    print('Ранг = ', CurrentUser.rang)
    print('Бесплатных операций в день = ', CurrentUser.FreeLimitPerDay)

    print()
    print('Балланс = ', CurrentUser.ballance)
    print('Последняя операция = ', CurrentUser.LastOperationDate)
    print('Всего операций = ', CurrentUser.OperationAll)
    print('Операций cегодня = ', CurrentUser.OperationToday)
    print('Бесплатных операций осталось = ', CurrentUser.FreeLimitToday)

    print()
    print('Иницализация = ', CurrentUser.GetFirstOperation)



## Демонстрация работы

In [713]:
# Создание пользователя
CurrentUser = User(name = 'dmagog',
                   email = 'georgy-mamarin@mail.ru',
                   startBallance = 5,
                   FreeLimitPerDay = 3)

CurrentUser

In [719]:
# Для демонстрации сделаем выбор модели случайным образом
if bool(random.randint(0, 1)): CurrModel = CurrModel_1
else: CurrModel = CurrModel_2

#Пользователь вызывает модель
CurrentUser.useModel(model = CurrModel, 
                     payment = CurrModel.price)


print()
print('Балланс = ', CurrentUser.ballance)
print('Последняя операция = ', CurrentUser.LastOperationDate)
print('Всего операций = ', CurrentUser.OperationAll)
print('Операций cегодня = ', CurrentUser.OperationToday)
print('Бесплатных операций осталось = ', CurrentUser.FreeLimitToday)

print()
print(CurrentUser.HistoryOperation)

| Модель сработала:  False |

Пополнение баланса на 2 ед. прошлошло успешно. Текущий балланс — 3 ед.

Балланс =  3
Последняя операция =  2025-03-24 15:56:35.236218
Всего операций =  4
Операций cегодня =  4
Бесплатных операций осталось =  0

                         Date      Operation Success
0  2025-03-24 15:56:33.856679  Запуск модели    True
1  2025-03-24 15:56:34.211091  Запуск модели   False
2  2025-03-24 15:56:34.711870  Запуск модели    True
3  2025-03-24 15:56:35.056205  Запуск модели    True
4  2025-03-24 15:56:35.235494  Запуск модели    True
5  2025-03-24 15:56:35.412161  Запуск модели   False


In [431]:
# Пополнить баланс
CurrentUser.refund(5)

Пополнение баланса на 5 ед. прошлошло успешно. Текущий балланс — 35 ед.


In [721]:
# Сбросить счетчик бесплатных генераций в день
CurrentUser.refillLimits()

Лимиты бесплатных вызовов в день обновлены. Сейчас доступно — 3


In [ ]:
showUserDetails(CurrentUser)

Имя пользователя =  dmagog
Эл. почта =  georgy-mamarin@mail.ru
Дата регистрации =  2025-03-24 15:56:31.881058
Роль =  0
Ранг =  0
Бесплатных операций в день =  3

Балланс =  3
Последняя операция =  2025-03-24 15:56:35.236218
Всего операций =  4
Операций cегодня =  4
Бесплатных операций осталось =  3

Иницализация =  True
